In [2]:
!pip install selenium
!pippip instalinstall webdriver
!pip install chromedriver_autoinstaller

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
/bin/bash: line 1: pippip: command not found


In [3]:
!pip install webdriver_manager

In [4]:
# Tải xuống và giải nén chromedriver
!wget https://storage.googleapis.com/chrome-for-testing-public/122.0.6261.69/linux64/chromedriver-linux64.zip
!unzip chromedriver-linux64.zip

# Di chuyển chromedriver vào /usr/local/bin/
!mv /content/chromedriver-linux64/chromedriver /usr/local/bin/


# Cấp quyền thực thi
!chmod +x /usr/local/bin/chromedriver

!chmod +x /usr/local/bin/chromedriver

!apt-get update
!apt-get install -y wget unzip
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt --fix-broken install -y


--2024-03-13 08:06:08--  https://storage.googleapis.com/chrome-for-testing-public/122.0.6261.69/linux64/chromedriver-linux64.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.193.207, 172.217.204.207, 172.217.203.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.193.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8612965 (8.2M) [application/zip]
Saving to: ‘chromedriver-linux64.zip’

chromedriver-linux6 100%[===================>]   8.21M  --.-KB/s    in 0.1s    

2024-03-13 08:06:08 (67.0 MB/s) - ‘chromedriver-linux64.zip’ saved [8612965/8612965]

Archive:  chromedriver-linux64.zip
  inflating: chromedriver-linux64/LICENSE.chromedriver  
  inflating: chromedriver-linux64/chromedriver  
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:4 https://cloud.r-

In [5]:
from datetime import datetime as dt
import time
from selenium import webdriver
from selenium.webdriver.support.select import Select
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
#Setup môi trường để chạy chromedriver trên colab
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

service = Service('/usr/local/bin/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Chế độ headless (không có giao diện đồ họa)
options.add_argument('--no-sandbox')

# Khởi tạo trình điều khiển Chrome
driver = webdriver.Chrome(service=service, options=options)
driver.quit()

In [7]:
# Thấy cái này thì chạy đi----------------------------------------------------------------------------------------------------


def Craping_link_booking(driver, accommodation_list, star_rating_hotel_list, area_info, review_list, hightlight_list, comment_user_list, voting_user_list):
    try:
        # Star rating hotel
        elements_xpath = '//span[@aria-hidden="true" and @class="fcd9eec8fb d31eda6efc c25361c37f"]'
        star_rating_hotel_list = driver.find_elements(By.XPATH, elements_xpath)

        # accommodation_list
        options = [
            '//*[@data-capla-component-boundary="b-property-web-property-page/PropertyHeaderName"]',
            '//*[@id="showMap2"]',
            '//ul[@class="c807d72881 d1a624a1cc e10711a42e"]'
        ]
        for i in range(len(options)):
            try:
                element = driver.find_element(By.XPATH, f'{options[i]}')
                # Lấy và in nội dung của phần tử
                info_ult = element.text.split('\n')
                accommodation_list.append(info_ult)
            except NoSuchElementException:
                # Handle the case when the element is not found
                accommodation_list.append(['Null'])

        # Area
        paired_elements = []
        try:
        # Tìm phần tử với XPath
            info = driver.find_elements(By.XPATH, '//ul[@data-location-block-list="true"]')
            # Xử lý dữ liệu nếu phần tử được tìm thấy
            ip = [i.text.split('\n') for i in info]
        except:
            # Xử lý trường hợp không tìm thấy phần tử
            ip = ['Null']

        if ip[0] != 'Null':
            for i in ip:
              for j in range(0, len(i), 2):
                paired_elements.append(i[j] + ": " + i[j+1])

        #Review hotel
        reviews = driver.find_elements(By.XPATH,'//p[@data-testid="property-description"]')
        for review in reviews:
            try:
                review_list.append(review.text)
            except Exception as e:
                review_list.append("No review")

        #Propertive highlights
        parent_elements = driver.find_elements(By.XPATH, '//div[@class="page-section hp--desc_highlights js-k2-hp--block"]')
        for parent in parent_elements:
            highlights = parent.find_elements(By.XPATH, './/span[@class="ph-item-copy "]')
            apartment_element = parent.find_elements(By.XPATH, './/span[@class="ph-item-copy-top-location"]')

            # Check if the apartment_element list is not empty before accessing the first element
            if apartment_element:
                apartment = apartment_element[0].text
            else:
                apartment = "N/A"  # or any other default value you want to assign

            top_item_value = ";".join(hig.text for hig in highlights)
            combined_value = '; '.join([top_item_value, apartment])

            if combined_value:      # Lấy nội dung của từng phần tử và thêm vào danh sách
                hightlight_list.append(combined_value)
            else:
                # Kiểm tra nếu danh sách rỗng, thêm "None" vào danh sách
                hightlight_list.append("None")

        cmt_elements = driver.find_elements(By.XPATH, '//div[@aria-label="Đây là một khung giới thiệu có các đường trượt xoay, hiển thị các đánh giá nổi bật về chỗ nghỉ. Hãy sử dụng các nút Tiếp theo và Trước đó để di chuyển giữa các hình ảnh."]')
        if not cmt_elements:
          print("Lỗi A")
          comment_user_list.append("No comment")
        else:
          for comment in cmt_elements:
              try:
                  # Re-find the cmt elements inside the loop to avoid StaleElementReferenceException
                  print("tới đây chưa")
                  cmt = comment.find_elements(By.XPATH, './/li[@aria-hidden="false"]')
                  for user in cmt:
                      try:
                          us = user.find_element(By.XPATH, './/div[@class="a3332d346a e6208ee469"]')
                          country = user.find_element(By.XPATH, './/span[@class="afac1f68d9 a1ad95c055"]')
                          fb = user.find_element(By.XPATH, './/div[@class="a53cbfa6de b5726afd0b"]')
                          comment_user_list.append(f"{us.text}({country.text}): {fb.text}")
                      except (StaleElementReferenceException, NoSuchElementException):
                          comment_user_list.append("No comment")
              except StaleElementReferenceException:
                  # Handle StaleElementReferenceException for the outer loop
                  print("Lỗi C")
                  pass
        #Vote hotel
        try:
            user_vote = driver.find_element(By.XPATH, '//div[@class="a3b8729ab1 d86cee9b25"]')
            voting_user_list.append(user_vote.text)
        except(NoSuchElementException):
            voting_user_list.append('0')

        return len(star_rating_hotel_list), accommodation_list[0], accommodation_list[1], accommodation_list[2], paired_elements, review_list, hightlight_list, comment_user_list, voting_user_list

    except Exception as e:
        # Handle other exceptions (generic exception)
        print(f"An error occurred: {e}")
        return 0, ['Null'], ['Null'], ['Null'], ['Null'], ['Null'], ['Null'], ['Null']


def crawl_link_booking(base_url):
    accommodation_list, star_rating_hotel_list, area_info, review_list, hightlight_list, comment_user_list, voting_user_list  = ([] for _ in range(7))
    star_rating_hotel_list, Name_Hotel, Locations, areas, paired_elements  = ([] for _ in range(5))
    service = Service('/usr/local/bin/chromedriver')
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Chế độ headless (không có giao diện đồ họa)
    options.add_argument('--no-sandbox')
    # Khởi tạo trình điều khiển Chrome
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(base_url)
    star_rating_hotel_list, Name_Hotel, Locations, areas, paired_elements, review_list, hightlight_list, comment_user_list, voting_user_list = Craping_link_booking(driver, accommodation_list, star_rating_hotel_list, area_info, review_list, hightlight_list, comment_user_list, voting_user_list)
    driver.quit()
    return star_rating_hotel_list, Name_Hotel, Locations, areas, paired_elements, review_list, hightlight_list, comment_user_list, voting_user_list

In [8]:
#Hotel Ha Noi
df_link = pd.read_csv("/content/LinkBookingBinhThuan.csv", index_col=0)
df_check = df_link.dropna(subset=['Link'])
df_check = df_check[:]
df_check[['star_rating_hotel_list', 'Name_Hotel', 'Locations', 'areas', 'paired_elements', 'review_list', 'hightlight_list', 'comment_user_list', 'voting_user_list']]=df_check['Link'].apply(crawl_link_booking).apply(pd.Series)

# Print the final DataFrame
df_check

tới đây chưa
Lỗi A
tới đây chưa
tới đây chưa
Lỗi A
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
Lỗi A
Lỗi A
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
Lỗi A
tới đây chưa
tới đây chưa
tới đây chưa
Lỗi A
tới đây chưa
Lỗi A
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
Lỗi A
Lỗi A
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
Lỗi A
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
Lỗi A
tới đây chưa
tới đây chưa
tới đây chưa
Lỗi A
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
Lỗi A
tới đây chưa
Lỗi A
Lỗi A
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
Lỗi A
Lỗi A
tới đây chưa
Lỗi A
Lỗi A
Lỗi A
Lỗi A
Lỗi A
Lỗi A
Lỗi A
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chưa
tới đây chư

,price,Link,star_rating_hotel_list,Name_Hotel,Locations,areas,paired_elements,review_list,hightlight_list,comment_user_list,voting_user_list
0,VND 4.824.040,https://www.booking.com/hotel/vn/apec-mui-ne-t...,4,[Apec Mandala Mũi Né Condotel],"[Đường tỉnh 716, Mũi Né, Việt Nam – Vị trí ...","[Hồ bơi ngoài trời, WiFi nhanh miễn phí (119 M...","[Bảo tàng Ngọc Trai Mũi Né: 16 km, Песикипесик...","[Nhìn ra thành phố, Apec Mandala Mũi Né Condot...",[Địa điểm hàng đầu: Được khách gần đây đánh gi...,[Nha(Hungary): “The whole complex was well pla...,"[8,4]"
1,VND 3.181.600,https://www.booking.com/hotel/vn/victor-group-...,5,[Apec Mandala Mui Ne],"[Đường Xuân Thủy, Ấp Thiẹn Ái, Việt Nam – Vị ...","[Hồ bơi trong nhà, Dịch vụ đưa đón sân bay (mi...","[Bảo tàng Ngọc Trai Mũi Né: 16 km, Песикипесик...","[Nằm ở Ấp Thiẹn Ái, cách Bãi biển Đồi Cát Đỏ 2...",[Địa điểm hàng đầu: Được khách gần đây đánh gi...,[No comment],"[9,0]"
2,VND 1.350.000,https://www.booking.com/hotel/vn/quoc-thang-ho...,1,[I Hostel Muine - A Little White Homestay By T...,"[188 Nguyễn Đình Chiểu, Ấp Thiện Long, Việt N...","[2 nhà hàng, Dịch vụ phòng, WiFi miễn phí...","[Bảo tàng Ngọc Trai Mũi Né: 2,9 km, Песикипеси...","[Với nhà hàng, quầy bar và nhìn rabiển, I Host...",[Địa điểm hàng đầu: Được khách gần đây đánh gi...,"[Truong(Việt Nam): “Lần 2 quay lại ,vẫn tốt v...","[9,4]"
3,VND 7.118.000,https://www.booking.com/hotel/vn/movenpick-res...,5,[Mövenpick Resort Phan Thiet],[Hon Gio - Thuan Quy Street Tien Hoa Hamlet Ti...,"[2 hồ bơi, Giáp biển, Phòng gia đình, Chỗ đ...","[Bảo Tàng Hồ Chí Minh: 10 km, Công Viên Nguyễn...","[Nằm ở Phan Thiết, Mövenpick Resort Phan Thiet...","[Kiểu lục địa, Thực đơn chay, Kiểu Á, Tự chọn;...",[Haanhnguyen2601(Việt Nam): “Thiết kế của res...,"[8,0]"
4,VND 2.380.000,https://www.booking.com/hotel/vn/m-continental...,5,[M Continental by Mandala Mui Ne],"[DT716, phuong Mui Ne, Phan Thiết, Việt Nam –...","[Hồ bơi ngoài trời, WiFi miễn phí, Phòng gi...","[Bảo tàng Ngọc Trai Mũi Né: 16 km, Песикипесик...","[Nằm ở Phan Thiết, cách Bãi biển Long Sơn 6 ph...",[Kiểu lục địa;Có bãi đậu xe riêng miễn phí ở k...,[No comment],"[7,1]"
...,...,...,...,...,...,...,...,...,...,...,...
495,VND 10.500.000,https://www.booking.com/hotel/vn/can-ho-2-phon...,0,[CĂN HỘ 2 PHÒNG NGỦ tại Apec Mandala Wyndham M...,"[Apec Mandala Khu nghỉ dưỡng Mũi Né, Ấp Thiẹn ...","[Hồ bơi, Chỗ đỗ xe miễn phí, WiFi tốt miễn...","[Bảo tàng Ngọc Trai Mũi Né: 15 km, Песикипесик...","[Nằm ở Ấp Thiẹn Ái, CĂN HỘ 2 PHÒNG NGỦ tại Ape...",[Có chỗ đậu xe miễn phí trong khuôn viên; N/A],[No comment],[0]
496,VND 13.500.000,https://www.booking.com/hotel/vn/biet-thu-nghi...,0,[Biệt thự nghỉ dưỡng 130m 3phòng ngủ tại Novaw...,"[Biệt thự số NWP 1.3-04.51, Bình Sum, Việt Na...","[Hồ bơi ngoài trời, Chỗ đỗ xe miễn phí, Wi...","[Bảo Tàng Hồ Chí Minh: 13 km, Công Viên Nguyễn...","[Với hồ bơi ngoài trời, khu vườn, cùng sân hiê...",[Có chỗ đậu xe riêng miễn phí trong khuôn viên...,[No comment],[0]
497,VND 990.000,https://www.booking.com/hotel/vn/cat-tien-gues...,3,[Cat Tien Guesthouse],"[59 Huynh Thuc Khang, Mũi Né, Việt Nam – Vị...","[WiFi miễn phí, Giáp biển, Phòng gia đình,...","[Bảo tàng Ngọc Trai Mũi Né: 4,6 km, Песикипеси...",[Cat Tien Guesthouse nằm cách Suối Tiên 50 m v...,[Địa điểm hàng đầu: Được khách gần đây đánh gi...,"[Khanh(Việt Nam): “Phòng sạch, đẹp, tiện nghi...","[8,6]"
498,VND 2.100.000,https://www.booking.com/hotel/vn/bong-homestay...,0,[Bống Homestay],"[80 Hồ Xuân Hương, Ấp Khánh Phước (1), Việt N...","[Chỗ đỗ xe miễn phí, WiFi miễn phí]","[Bảo tàng Ngọc Trai Mũi Né: 9 km, Песикипесики...","[Tọa lạc ở Ấp Khánh Phước (1), cách Bãi biển M...",[Có chỗ đậu xe riêng miễn phí trong khuôn viên...,[No comment],[0]


In [9]:
fields_to_convert = ['Name_Hotel', 'Locations', 'areas', 'paired_elements', 'review_list', 'hightlight_list', 'comment_user_list', 'voting_user_list']

# Chuyển các giá trị từ list sang string và loại bỏ chúng ra khỏi list
for field in fields_to_convert:
    df_check[field] = df_check[field].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# Hiển thị DataFrame mới
print(df_check)
df_check.to_csv("/content/drive/MyDrive/Colab Notebooks/BinhThuanDataset.csv", encoding='utf-8', index=False)

              price                                               Link  \
0     VND 4.824.040  https://www.booking.com/hotel/vn/apec-mui-ne-t...   
1     VND 3.181.600  https://www.booking.com/hotel/vn/victor-group-...   
2     VND 1.350.000  https://www.booking.com/hotel/vn/quoc-thang-ho...   
3     VND 7.118.000  https://www.booking.com/hotel/vn/movenpick-res...   
4     VND 2.380.000  https://www.booking.com/hotel/vn/m-continental...   
..              ...                                                ...   
495  VND 10.500.000  https://www.booking.com/hotel/vn/can-ho-2-phon...   
496  VND 13.500.000  https://www.booking.com/hotel/vn/biet-thu-nghi...   
497     VND 990.000  https://www.booking.com/hotel/vn/cat-tien-gues...   
498   VND 2.100.000  https://www.booking.com/hotel/vn/bong-homestay...   
499   VND 5.700.000  https://www.booking.com/hotel/vn/can-ho-ocean-...   

     star_rating_hotel_list  \
0                         4   
1                         5   
2                 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
